# Interact with `SampleGen` simulation

requires litex_server running and connected to the VC707 USB-uart port:

```bash
# both run at the same time in separate windows ...
$ python3 sim_sample_gen.py
$ litex_server --uart --uart-port socket://localhost:1111 --bind-port 2234
```

In [1]:
import numpy as np
from time import sleep
from litex import RemoteClient
import sys
sys.path.append('../spi')
from client_tools import getId, hd, big_write, setSamples, set_trigger_freq

In [2]:
# Connect to litex server
r = RemoteClient(csr_csv='out/csr.csv', debug=False, port=2234)
r.open()
getId(r)

'LiteX Simulation 2021-04-09 17:57:52'

In [3]:
set_trigger_freq(r, 1e6)

In [4]:
samples = np.arange(0x100)
setSamples(r, samples)

In [5]:
assert r.regs.sample_gen_wfm_len.read() == len(samples)//16 -1

In [6]:
r.regs.sample_gen_wfm_len.read()

15

In [7]:
r.regs.sample_gen_trig_cnt_max.read()

312

In [11]:
r.regs.sample_gen_enable_ext_trig.write(1)

In [13]:
# start vcd dump for a few ms, then stop sim.
r.regs.sim_trace_enable.write(1)
sleep(0.01)
r.regs.sim_trace_enable.write(0)
# r.regs.sim_finish_finish.write(1)

to inspect result, run

```bash
gtkwave out/gateware/sim.vcd
```

look for `TOP/sim/source_converter0` which is the sample stream for the DAC.